# Imports

In [1]:
import pydot
import graphviz
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [2]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, merge, concatenate, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG
from keras.layers import dot
from tensorflow.keras.utils import plot_model

2021-08-31 10:49:19.486222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 10:49:19.486271: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

# KNN prediction 

In [5]:
# imports

## dependencies

import joblib
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pickle


import pydot
import graphviz
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, merge, concatenate, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG
from keras.layers import dot
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

#!pip install scikit-learn==0.24.2

## files

# model_path = "../data/models_anime_map_knn_model.joblib"
# pickle.load(open('../model.sav', 'rb'))

def get_anime():
    anime_df_relevant_PG = pd.read_csv("../data/anime_df_relevant_PG.csv")
    return anime_df_relevant_PG.rename(columns={'MAL_ID' : 'anime_id'})

def get_data():
    data = pd.read_csv('../data/processed_data/active_users_df_10PlusRatings_partial.csv')
    return data

def get_model():
    return pickle.load(open('../model_knn.sav', 'rb'))

get_model()

def process_data():
    data_users_df = get_data()
    data_users_df['rating'] = data_users_df['rating']/10
    
    anime_df_relevant_PG = get_anime()
    anime_name_df = anime_df_relevant_PG[['anime_id','Name']]
    data_users_df_merge = data_users_df.merge(anime_name_df, on = 'anime_id', how='inner')
    pivot_df = data_users_df_merge.pivot_table(index='anime_id',columns='user_id',values='rating').fillna(0)
    
    anime_Genres_df = anime_df_relevant_PG[['anime_id','Genres']]
    anime_Genres_df_encoded = pd.concat(objs = [anime_Genres_df.drop(columns = 'Genres', axis =1), anime_Genres_df['Genres'].str.get_dummies(sep=", ")], axis = 1)
    anime_Genres_df_encoded = anime_Genres_df_encoded.set_index('anime_id')
    
    pivot_df = pivot_df.merge(anime_Genres_df_encoded, how='inner',left_index=True, right_index=True)
    anime_name_pivot_df = data_users_df_merge[['anime_id','Name']].drop_duplicates()
    anime_name_pivot_df = anime_name_pivot_df.sort_values('anime_id')
    anime_name_pivot_df = anime_name_pivot_df.reset_index().drop(columns = 'index')
    
    return pivot_df, anime_name_pivot_df

# predict

def recomendation_10PlusRatings(anime_name, nb_recomendation = 10):
    pivot_df, anime_name_pivot_df = process_data()
    model = get_model()
    index_nb = anime_name_pivot_df.index[anime_name_pivot_df['Name'] == anime_name].tolist()[0]
    distances, indices = model.kneighbors(pivot_df.iloc[index_nb,:].values.reshape(1, -1), n_neighbors = nb_recomendation + 1)

    prediction = []
    for i in range(0, len(distances.flatten())):
        if i == 0:
            prediction.append([pivot_df.index[indices.flatten()[i]],0])
        else:
            prediction.append([pivot_df.index[indices.flatten()[i]],distances.flatten()[i]])
    results = []
    for i in range(len(prediction)):
        anime_name = anime_name_pivot_df.query(f'anime_id == {prediction[i][0]}').iloc[0].Name
        distance = prediction[i][1]
        results.append([anime_name,distance])
    return results

# Test dataset preparation before feeding to NeuMF

 - concat lists of recom
 - remove duplicates
 - add user_id
 - map anime name and its number to feed to model
 - put in model
 - predict probable user ratings for predicted anime from knn
 - sort by desc

In [6]:
knn_recommend = pd.DataFrame(recomendation_10PlusRatings('Naruto'))[0]

In [7]:
knn_recommend

0                               Naruto
1                   Naruto: Shippuuden
2                           Death Note
3                               Bleach
4                   Shingeki no Kyojin
5     Fullmetal Alchemist: Brotherhood
6      Code Geass: Hangyaku no Lelouch
7                     Sword Art Online
8                            One Piece
9                           Fairy Tail
10                       One Punch Man
Name: 0, dtype: object

## Transorm list of recom into dataset suitable for NeuMF, column of user number(unique) and anime names

In [8]:
columns_user_id = np.zeros(len(knn_recommend))

In [9]:
# load anime dataset to map anime name from recom to its anime_id to feed to neuMF
def get_anime(): 
    anime_df_relevant_PG = pd.read_csv("../data/anime_df_relevant_PG.csv") 
    return anime_df_relevant_PG.rename(columns={'MAL_ID' : 'anime_id'})

In [10]:
#np.full(len(knn_recommend),fill_value = 0)

In [11]:
test_df_to_neuMF = 
df3 = pd.DataFrame(test_to_neuMF, columns=['user_id', 'anime_id'])

SyntaxError: invalid syntax (2781966839.py, line 1)

In [33]:
def load_model():
    model = tf.keras.models.load_model('../neuMFmodel.h5')
    #model = joblib.load('neuMFmodel.joblib')
    return model


In [34]:
model_neuMF = load_model()

In [ ]:
def predict(test,model):
    y_pred = np.round(model.predict([test.user_id, test.anime_id]), decimals=2)
    y_true = test.rating
    mean_absolute_error(y_true, y_pred)
    return y_true,y_pred